In [1]:
import numpy as np
np.random.seed(0)
import pandas as pd
import os
from datetime import datetime
import time
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, maxabs_scale
from math import pi
from numpy import sin, cos, linspace
import random
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
from sklearn.metrics import roc_auc_score, mean_squared_error

In [3]:
sample_no = 1
data_dir = "./ncmf_sim_data"
#list_dataset_names = ["dt1"]
#list_dataset_names = ["ds1","ds2","ds3"]
list_dataset_names = ['dt1', 'ds1', 'ds2', 'ds3', 'dn1', 'dn2', 'dn3']

In [4]:
matrix_types = {
    "real": ["X0","X1","X3"],
    "binary": ["X2","X4"]
}

In [5]:
list_test_mat_ids = ["X0","X2"]
list_test_mat_idx = [0,2]
#
# list_test_mat_ids = ["X0"]
# list_test_mat_idx = [0]

In [6]:
def get_perf_metrics(sample_no,\
                        data_dir,\
                        list_dataset_names,\
                        matrix_types,\
                        list_test_mat_ids,\
                        list_test_mat_idx
                        ):
    dict_dname_perf = {}
    out_dir_base = data_dir + "/cmf/out/"
    #
    for dataset_name in list_dataset_names:
        print("dataset_name: ",dataset_name)
        dict_dname_perf[dataset_name] = {}
        #
        out_dir_recons = out_dir_base + dataset_name + "/"
        data_dir_gt = data_dir + "/" + dataset_name + "/"
        #
        sampled_dict_id_idx_file = data_dir_gt + f"sampled{sample_no}_dict_id_idx.pkl"
        dict_id_idx = pkl.load(open(sampled_dict_id_idx_file,"rb"))
        #
        for i in np.arange(len(list_test_mat_idx)):       
            test_mat_idx = list_test_mat_idx[i]
            test_mat_id = list_test_mat_ids[i]
            #
            fname_pred = out_dir_recons + str(int(test_mat_idx)+1) + "_pred.csv"
            X_pred = pd.read_csv(fname_pred,header=None).to_numpy()
            #
            fname_list_gt = data_dir_gt + "sampled1_link.dat.test."+str(test_mat_idx)
            df_list_gt = pd.read_csv(fname_list_gt,sep="\t",header=None)
            df_list_gt.columns = ["i_idx","j_idx","value"]
            #
            list_gt_vals = []
            list_pred_vals = []
            for idx,row in df_list_gt.iterrows():
                i = int(dict_id_idx[row["i_idx"]])
                j = int(dict_id_idx[row["j_idx"]])
                #
                cur_val_gt = row["value"]
                list_gt_vals.append(cur_val_gt)
                #
                cur_val_pred = X_pred[i,j]
                list_pred_vals.append(cur_val_pred)
            #
            print("mat_idx:",test_mat_idx,", list_gt_vals - min: ",np.min(list_gt_vals),", max: ",np.max(list_gt_vals))
            print("mat_idx:",test_mat_idx,", list_pred_vals - min: ",np.min(list_pred_vals),", max: ",np.max(list_pred_vals))
            #
            if test_mat_id in matrix_types["real"]:
                eval_mat_dtype = "real"
            elif test_mat_id in matrix_types["binary"]:
                eval_mat_dtype = "binary"
            else:
                assert False    
            #
            if eval_mat_dtype == "real":
                cur_perf = mean_squared_error(list_gt_vals,list_pred_vals)
                cur_perf_metric = "mse"
            elif eval_mat_dtype == "binary":
                cur_perf = roc_auc_score(list_gt_vals,list_pred_vals)
                cur_perf_metric = "auc"
            else:
                assert False
            #
            dict_dname_perf[dataset_name][test_mat_idx] = {
                "perf":cur_perf,
                "perf_metric":cur_perf_metric
            }
        print("###")
    #
    pkl.dump(dict_dname_perf,open(out_dir_base+"dict_perf_cmf.pkl","wb"))
    return dict_dname_perf

In [7]:
dict_perf = get_perf_metrics(sample_no,\
                        data_dir,\
                        list_dataset_names,\
                        matrix_types,\
                        list_test_mat_ids,\
                        list_test_mat_idx)

dataset_name:  dt1
mat_idx: 0 , list_gt_vals - min:  -0.5879045133966941 , max:  0.9988405659720564
mat_idx: 0 , list_pred_vals - min:  -0.9196351479116031 , max:  1.2545225175393602
mat_idx: 2 , list_gt_vals - min:  0.0 , max:  1.0
mat_idx: 2 , list_pred_vals - min:  -4827.90828453435 , max:  4810.21456571804
###
dataset_name:  ds1
mat_idx: 0 , list_gt_vals - min:  0.0 , max:  0.9985113374128098
mat_idx: 0 , list_pred_vals - min:  -0.594391609254524 , max:  1.14282802902824
mat_idx: 2 , list_gt_vals - min:  0.0 , max:  1.0
mat_idx: 2 , list_pred_vals - min:  -5245.4614911594 , max:  4644.093623378561
###
dataset_name:  ds2
mat_idx: 0 , list_gt_vals - min:  0.0 , max:  0.998449431413366
mat_idx: 0 , list_pred_vals - min:  -0.5859233791706899 , max:  1.18524526602001
mat_idx: 2 , list_gt_vals - min:  0.0 , max:  1.0
mat_idx: 2 , list_pred_vals - min:  -5733.0179959670895 , max:  5798.703845166179
###
dataset_name:  ds3
mat_idx: 0 , list_gt_vals - min:  0.0 , max:  0.9997617218829612
mat

In [8]:
pd.DataFrame(dict_perf)

,dt1,ds1,ds2,ds3,dn1,dn2,dn3
0,"{'perf': 0.25969109929779954, 'perf_metric': '...","{'perf': 0.18034964750163093, 'perf_metric': '...","{'perf': 0.1934823640844111, 'perf_metric': 'm...","{'perf': 0.19931691163859663, 'perf_metric': '...","{'perf': 0.3267291557876415, 'perf_metric': 'm...","{'perf': 0.29587837115488774, 'perf_metric': '...","{'perf': 0.33432251914809996, 'perf_metric': '..."
2,"{'perf': 0.505564865555972, 'perf_metric': 'auc'}","{'perf': 0.5019045254008069, 'perf_metric': 'a...","{'perf': 0.49709357253519787, 'perf_metric': '...","{'perf': 0.4849790896242624, 'perf_metric': 'a...","{'perf': 0.49839230286607944, 'perf_metric': '...","{'perf': 0.49453125272356063, 'perf_metric': '...","{'perf': 0.5030012654412225, 'perf_metric': 'a..."


In [9]:
dict_perf

{'dt1': {0: {'perf': 0.25969109929779954, 'perf_metric': 'mse'},
  2: {'perf': 0.505564865555972, 'perf_metric': 'auc'}},
 'ds1': {0: {'perf': 0.18034964750163093, 'perf_metric': 'mse'},
  2: {'perf': 0.5019045254008069, 'perf_metric': 'auc'}},
 'ds2': {0: {'perf': 0.1934823640844111, 'perf_metric': 'mse'},
  2: {'perf': 0.49709357253519787, 'perf_metric': 'auc'}},
 'ds3': {0: {'perf': 0.19931691163859663, 'perf_metric': 'mse'},
  2: {'perf': 0.4849790896242624, 'perf_metric': 'auc'}},
 'dn1': {0: {'perf': 0.3267291557876415, 'perf_metric': 'mse'},
  2: {'perf': 0.49839230286607944, 'perf_metric': 'auc'}},
 'dn2': {0: {'perf': 0.29587837115488774, 'perf_metric': 'mse'},
  2: {'perf': 0.49453125272356063, 'perf_metric': 'auc'}},
 'dn3': {0: {'perf': 0.33432251914809996, 'perf_metric': 'mse'},
  2: {'perf': 0.5030012654412225, 'perf_metric': 'auc'}}}